# Applying LoRAs to Lifting Self-Attention Layers

We would like to show that adding LoRAs (Low Rank Adaptations) to the lifting self-attention layer does not disrupt equivariance. For more information see [Group Equivariant Stand-Alone Self-Attention For Vision](https://openreview.net/forum?id=JkfYjnOEo6M&noteId=dxKhFZNxn-D) for equivariance, and [LoRA: Low-Rank Adaptation of Large Language Models](https://openreview.net/forum?id=nZeVKeeFYf9) for details on LoRAs. 


For the lifting self-attention layer to be $G$-equivariant, we must have that $m_{G \uparrow}^r[L_g[f], \rho](i, h) = [m_{G \uparrow}^r[f, \rho]](i, h)$. Consider a $g$-transformed input signal $L_g[f](i) = L_yL_{h_3}[f](i) = f(x^{-1}(h_3^{-1}(x(i)-y)))$. Here $g = (y, h_3) \in \mathbb{R}^{d_{in}} \rtimes \mathcal{H}$. The lifting self-attention layer with LoRAs, applied to $L_g[f]$ is:

\begin{align}
&m_{G \uparrow}^{r, LoRA}[L_yL_{h_3}[f], \rho](i, h) \\
&= \varphi_{out}\Bigg( \bigcup_{head \in [H]} \sum_{j \in N(i)} \sigma_j \Bigg( \langle \varphi_{qry}^{head}(L_yL_{h_3}[f](i)) + \Delta\varphi_{qry}^{head}(L_yL_{h_3}[f](i)), \varphi_{key}^{head}(L_yL_{h_3}[f](j)\\
&\quad + L_h[\rho](i, j)) + \Delta\varphi_{key}^{head}(L_yL_{h_3}[f](j) + L_h[\rho](i, j)) \rangle \Bigg) (\varphi_{val}^{head}(L_yL_{h_3}[f](j)) + \Delta\varphi_{val}^{head}(L_yL_{h_3}[f](j))) \Bigg)\\
&= \varphi_{out}\Bigg( \bigcup_{head \in [H]} \sum_{j \in N(i)} \sigma_j \Bigg( \langle \varphi_{qry}^{head}(f(x^{-1}(h_3^{-1}(x(i)-y))) + \Delta\varphi_{qry}^{head}(f(x^{-1}(h_3^{-1}(x(i)-y))), \varphi_{key}^{head}(f(x^{-1}(h_3^{-1}(x(j)-y)))\\
&\quad + L_h[\rho](i, j) + \Delta\varphi_{key}^{head}(f(x^{-1}(h_3^{-1}(x(j)-y))) + L_h[\rho](i, j)) \rangle \Bigg) (\varphi_{val}^{head}(f(x^{-1}(h_3^{-1}(x(j)-y)))) + \Delta\varphi_{val}^{head}(f(x^{-1}(h_3^{-1}(x(j)-y)))) \Bigg)\\
&= \varphi_{out}\Bigg( \bigcup_{head \in [H]} \sum_{x^{-1}(h_3^{-1}x(\overline{j})+y) \in N(x^{-1}(h_3^{-1}x(\overline{i})+y))} \sigma_j \Bigg( \langle \varphi_{qry}^{head}(f(\overline{i})) + \Delta\varphi_{qry}^{head}(f(\overline{i})), \varphi_{key}^{head}(f(\overline{j}))\\
&\quad + L_h[\rho](x^{-1}(h_3x(\overline{i})+y), x^{-1}(h_3x(\overline{j})+y)) + \Delta\varphi_{key}^{head}(f(\overline{j}) + L_h[\rho](x^{-1}(h_3x(\overline{i})+y), x^{-1}(h_3x(\overline{j})+y))) \rangle \Bigg) (\varphi_{val}^{head}(f(\overline{j})) + \Delta\varphi_{val}^{head}(f(\overline{j}))) \Bigg)\\
\end{align}

Here we have used $\overline{i} = x^{-1}(h_3^{-1}(x(i)-y)) \implies i = x^{-1}(h_3x(\overline{i})+y)$ and $\overline{j} = x^{-1}(h_3^{-1}(x(j)-y)) \implies j = x^{-1}(h_3x(\overline{j})+y)$. By using the defintion of $\rho(i, j)$ we can further reduce the expression:

\begin{align}
&= \varphi_{out} \Bigg( \bigcup_{h \in [H]} \sum_{x^{-1}(h_3^{-1}x(\overline{j})+y) \in N(x^{-1}(h_3^{-1}x(\overline{i})+y))} \sigma_{x^{-1}(h_3^{-1}x(\overline{j})+y)} \Bigg( \langle \varphi_{qry}^{(h)}f(\overline{i}) + \Delta\varphi_{qry}^{(h)}f(\overline{i}) , \varphi_{key}^{(h)}(f(\overline{j})\\
&\quad + \rho^P(h^{-1}(h_3^{-1}x(\overline{j})+y)-h^{-1}(h_3x(\overline{i})+y))) + \Delta\varphi_{key}^{(h)}(f(\overline{j}) + \rho^P(h^{-1}(h_3^{-1}x(\overline{j})+y)-h^{-1}(h_3x(\overline{i})+y))) \rangle \Bigg) (\varphi_{val}^{(h)}(f(\overline{j})) + \Delta\varphi_{val}^{(h)}(f(\overline{j}))) \Bigg)\\
&= \varphi_{out} \Bigg( \bigcup_{h \in [H]} \sum_{x^{-1}(h_3^{-1}x(\overline{j})+y) \in N(x^{-1}(h_3^{-1}x(\overline{i})+y))} \sigma_{x^{-1}(h_3^{-1}x(\overline{j})+y)} \Bigg( \langle \varphi_{qry}^{(h)}f(\overline{i}) + \Delta\varphi_{qry}^{(h)}f(\overline{i}), \varphi_{key}^{(h)}(f(\overline{j})\\ 
&\quad + \rho^P(h^{-1}h_3(x(\overline{j}) - x(\overline{i})))) + \Delta\varphi_{key}^{(h)}(f(\overline{j}) + \rho^P(h^{-1}h_3(x(\overline{j}) - x(\overline{i})))) \rangle \Bigg) (\varphi_{val}^{(h)}(f(\overline{j})) + \Delta\varphi_{val}^{(h)}(f(\overline{j}))) \Bigg)\\
&= \varphi_{out} \Bigg( \bigcup_{h \in [H]} \sum_{x^{-1}(h_3^{-1}x(\overline{j})+y) \in N(x^{-1}(h_3^{-1}x(\overline{i})+y))} \sigma_{x^{-1}(h_3^{-1}x(\overline{j})+y)} \Bigg( \langle \varphi_{qry}^{(h)}f(\overline{i}) , \varphi_{key}^{(h)}(f(\overline{j})\\
&\quad + L_{h_3^{-1}h})(\overline{i}, \overline{j}) + \Delta\varphi_{key}^{(h)}(f(\overline{j}) + L_{h_3^{-1}h})(\overline{i}, \overline{j})) \rangle \Bigg) (\varphi_{val}^{(h)}(f(\overline{j})) + \Delta\varphi_{val}^{(h)}(f(\overline{j}))) \Bigg)
\end{align}

Since unimodular (or compact) groups the area of summation remains equal for any $g \in G$, we have:

\begin{equation}
\sum_{x^{-1}(h_3x(\overline{j})+y) \in N(x^{-1}(h_3x(\overline{i})+y))}[\bullet] = \sum_{x^{-1}(h_3x(\overline{j})) \in N(x^{-1}(h_3x(\overline{i})))}[\bullet] = \sum_{x^{-1}(x(\overline{j})) \in N(x(\overline{i})))}[\bullet] = \sum_{\overline{j} \in N(\overline{i})}[\bullet]
\end{equation}

Resultantly, we can further reduce the expression above as:

\begin{align}
&m_{G \uparrow}^{r, LoRA}[L_yL_{h_3}[f], \rho](i, h) \\
&= \varphi_{out}\Bigg( \bigcup_{h \in [H]} \sum_{\overline{j} \in N(\overline{i})} \sigma_{\overline{j}}\Bigg(\langle \varphi_{qry}^{(h)}(f(\overline{i})) + \Delta\varphi_{qry}^{(h)}(f(\overline{i})), \varphi_{key}^{(h)}(f(\overline{i}) + L_{h_3^{-1}h}[\rho](\overline{i}, \overline{j})) + \Delta\varphi_{key}^{(h)}(f(\overline{i}) + L_{h_3^{-1}h}[\rho](\overline{i}, \overline{j}))  \rangle\Bigg) (\varphi_{val}^{(h)}(f(\overline{j}))+ \Delta\varphi_{val}^{(h)}(f(\overline{j}))) \Bigg)\\
&= m_{G \uparrow}^{r, LoRA}[r, \rho](\overline{i}, h_3^{-1}h)\\
&= m_{G \uparrow}^{r, LoRA}(x^{-1}(h_3^{-1}(x(i)-y)), h_3^{-1}h)\\
&= L_yL_{h_3}[m_{G \uparrow}^{r, LoRA}[f, \rho]](i, h)
\end{align}

So, we see that adding LoRAs does not disrupt equivariance of the liftin self-attention layer. 